### Leia o arquivo dos dados da Receita Federal e cruze eles com os dados da CEAP

In [16]:
import requests
import pandas as pd

url_camara = "http://www.camara.leg.br/cotas/Ano-2019.csv.zip"
url_receita = "https://serenata-de-amor-data.nyc3.digitaloceanspaces.com/2019-11-19-companies.csv.xz"

df_camara_2019 = pd.read_csv(url_camara, compression='zip', sep=';')
df_receita_2019 = pd.read_csv(url_receita, sep=',', dtype={'cnpj': str})

In [17]:
df_camara_2019["txtCNPJCPF"] = df_camara_2019["txtCNPJCPF"].str.replace(r'\D', '', regex=True)


In [22]:
df_merge_camara_receita = pd.merge(df_camara_2019, df_receita_2019, left_on='txtCNPJCPF', right_on='cnpj', how='left')
df_merge_camara_receita[['txtCNPJCPF', 'cnpj']]

,txtCNPJCPF,cnpj
0,03713903000108,03713903000108
1,03713903000108,03713903000108
2,03713903000108,03713903000108
3,03713903000108,03713903000108
4,03713903000108,03713903000108
...,...,...
479100,00722772700,NaN
479101,02012862000160,02012862000160
479102,02012862000160,02012862000160
479103,00000000000006,NaN


### Some o total da coluna `vlrLiquido` agrupando por `txtCNPJCPF` e `txtFornecedor` 

In [23]:
df_merge_camara_receita.groupby(["txtCNPJCPF", "txtFornecedor"])["vlrLiquido"].sum().sort_values(ascending=False)

txtCNPJCPF      txtFornecedor                                      
02012862000160  Cia Aérea - TAM                                        49845733.04
07575651000159  Cia Aérea - GOL                                        44014082.60
09296295000160  Cia Aérea - AZUL                                       12881535.32
07319323000191  PANTANAL VEÍCULOS LTDA                                  4362549.04
02012862000160  TAM LINHAS AEREAS S/A.                                  2344253.56
                                                                          ...     
04506629000169  NOVA ALIANÇA ATACADO LTDA - EPP 2                             4.59
25453685000152  BRUNA BANDEIRA FERRARI ME                                     4.00
04699200000135  W P PARK ESTACIONAMENTO LTDA ME                               3.60
18572225000269  Concessionária das Rodovias Centrais do Brasil  S/A           3.30
30159008000166  PAPELARIA E BAZAR SANTA LUZIA                                 2.20
Name: vlrLiquido, L

### Ordene os dados da consulta anterior de forma decrescente

In [31]:
df_merge_camara_receita.groupby(["txtCNPJCPF", "txtFornecedor"])["vlrLiquido"].sum().sort_values(ascending=False).head(20)

txtCNPJCPF      txtFornecedor                                        
02012862000160  Cia Aérea - TAM                                          49845733.04
07575651000159  Cia Aérea - GOL                                          44014082.60
09296295000160  Cia Aérea - AZUL                                         12881535.32
07319323000191  PANTANAL VEÍCULOS LTDA                                    4362549.04
02012862000160  TAM LINHAS AEREAS S/A.                                    2344253.56
02575829000148  Cia Aérea - AVIANCA                                       2275601.26
23721017000133  T2 COMUNICACAO, VIDEO E PRODUCOES EIRELI - EPP            2194668.00
02558157000162  TELEFÔNICA BRASIL S.A. VIVO                               1721626.30
05443449000148  VERTICE - SOCIEDADE CIVIL DE PROFISSIONAIS ASSOCIADOS     1419734.00
07575651000159  GOL LINHAS AEREAS S.A.                                    1236629.60
28800786000140  GP COBERTURA JORNALISTICA LTDA ME                         102700

### Consulte se houveram gastos reembolsados associados a empresas que na época estavam com status `08`


status == 8 significa `BAIXADA` (*a empresa foi extinta, seja por solicitação dos sócios ou por ação da Receita*)

In [51]:
df_merge_camara_receita_baixadas = df_merge_camara_receita[df_merge_camara_receita["situation"] == 8]
df_merge_camara_receita_baixadas

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,simples,size,administrative_person_category,name,situation,special_situation,phone,type,state,last_updated
1375,Danilo Forte,1.213373e+10,62881.0,91.0,2015,CE,PSDB,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0.0,1.0,49.0,GASTRO 3 BAR E RESTAURANTE LTDA,8.0,NaN,NaN,NaN,SP,NaN
7078,Celso Russomanno,1.252959e+09,73441.0,347.0,2015,SP,PRB,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,...,0.0,5.0,49.0,AUTO POSTO ORIBA LTDA,8.0,NaN,NaN,NaN,SP,NaN
11008,Henrique Fontana,3.341052e+10,73482.0,498.0,2015,RS,PT,55,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,0.0,1.0,50.0,DENIS RIBEIRO SOARES,8.0,NaN,NaN,NaN,RS,NaN
14597,Pompeo de Mattos,2.834689e+10,73486.0,516.0,2019,RS,PDT,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0.0,1.0,50.0,JOSE TERTULIANO MACIEL,8.0,NaN,NaN,NaN,RS,NaN
14598,Pompeo de Mattos,2.834689e+10,73486.0,516.0,2019,RS,PDT,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0.0,1.0,50.0,JOSE TERTULIANO MACIEL,8.0,NaN,NaN,NaN,RS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462391,Professora Dayane Pimentel,2.159738e+09,204565.0,210.0,2019,BA,UNIÃO,56,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0.0,5.0,5.0,NET SERVICOS DE COMUNICACAO S/A,8.0,NaN,NaN,NaN,BA,NaN
469782,Gil Cutrim,8.040588e+10,204575.0,77.0,2019,MA,REPUBLICANOS,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0.0,5.0,49.0,RESTAURANTE MADERO BRASILIA LTDA,8.0,NaN,NaN,NaN,DF,NaN
469783,Gil Cutrim,8.040588e+10,204575.0,77.0,2019,MA,REPUBLICANOS,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0.0,5.0,49.0,RESTAURANTE MADERO BRASILIA LTDA,8.0,NaN,NaN,NaN,DF,NaN
477253,Enéias Reis,9.450581e+10,207309.0,533.0,2019,MG,UNIÃO,56,9,PASSAGEM AÉREA - REEMBOLSO,...,0.0,5.0,16.0,AZUL LINHAS AEREAS BRASILEIRAS S.A.,8.0,NaN,NaN,NaN,DF,NaN


### Desafio extra

Esse desafio traz algumas ideias de análises. Fique a vontade para tentar encontrar essas respostas :)

- Qual o total gasto nos finais de semana?

- Agrupando por parlamentar e filtrando por finais de semana, qual o parlamentar que mais gastou em um final de semana?

- Tente filtrar os reembolsos envolvendo alimentação, agrupar por fornecedor, e verificar os fornecedores mais frequentados. Poderíamos analisar se existiram gastos com alto desvio padrão considerando os reembolsos de um restaurante em específico. Se você tem o grupo de gastos feitos em um local X, é possível encontrar a média de gasto feito por deputados nesse local X, e de repente encontrar qual teve um gasto muito acima desse valor. 

- Podemos tentar encontrar se houve parlamentar que teve alta frequência de reembolsos associados a empresas que já estavam com status `Baixada`.

In [39]:
df_merge_camara_receita["datEmissao"] = pd.to_datetime(df_merge_camara_receita.datEmissao)
df_merge_camara_receita["datEmissao"] = pd.to_datetime(df_merge_camara_receita.datEmissao, format="%Y-%m-%d")
df_merge_camara_receita_fds = df_merge_camara_receita[df_merge_camara_receita.datEmissao.dt.dayofweek > 5]
df_merge_camara_receita_fds["vlrDocumento"].sum()


12450742.83

In [41]:
df_merge_camara_receita_fds.groupby("txNomeParlamentar")["vlrDocumento"].sum().sort_values(ascending=False).head(10)

txNomeParlamentar
Professora Marcivania     105766.10
Natália Bonavides          85368.18
Shéridan                   77287.05
Gelson Azevedo             70167.48
Celina Leão                69671.10
Haroldo Cathedral          69431.78
Milton Vieira              69378.04
Silvia Cristina            65322.62
Euclydes Pettersen         64882.78
Frei Anastacio Ribeiro     61563.76
Name: vlrDocumento, dtype: float64

In [49]:
df_merge_camara_receita_alimentacao = df_merge_camara_receita[df_merge_camara_receita["txtDescricao"].str.contains("ALIMENTAÇÃO")]
#df_merge_camara_receita_alimentacao = df_merge_camara_receita[df_merge_camara_receita["txtDescricao"] == "FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR"]
df_merge_camara_receita_alimentacao.groupby("txtFornecedor")["vlrDocumento"].sum().sort_values(ascending=False)

txtFornecedor
SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC    401547.76
SILT SELF SERVICE EIRELI ME                         216737.06
RAFAEL DOS SANTOS SILVA 72238208100                 101368.30
M LELIS CAFETERIA LTDA ME                            43222.78
SILT SELF SERVICE EIRELI                             30625.14
                                                      ...    
Porto Vanila Café                                        6.83
SOLAR HOTEIS E TURISMO LTDA                              6.00
HOTEL 44 IJUI LTDA                                       6.00
CAFÉ E BAR PORTO ALEGRE LTDA                             5.00
BRUNA BANDEIRA FERRARI ME                                4.00
Name: vlrDocumento, Length: 3458, dtype: float64

In [52]:
df_merge_camara_receita_baixadas.groupby("txNomeParlamentar")["vlrDocumento"].sum().sort_values(ascending=False).head(10)

txNomeParlamentar
Gleisi Hoffmann        62065.18
Alex Manente           31150.00
FREIRE JÚNIOR          25000.00
Luciano Bivar          21000.00
TAKAYAMA               17000.00
MARCONDES GADELHA      17000.00
Gilberto Nascimento    14000.00
Carlos Veras           11508.60
INDIO DA COSTA         10504.60
Bosco Costa            10000.00
Name: vlrDocumento, dtype: float64

In [54]:
df_merge_camara_receita_baixadas.groupby(["txNomeParlamentar", "txtFornecedor", "txtDescricao"])["vlrDocumento"].sum().sort_values(ascending=False).head(10)

txNomeParlamentar    txtFornecedor                                 txtDescricao                                 
Gleisi Hoffmann      GOL LINHAS AÉREAS INTELIGENTES                PASSAGEM AÉREA - REEMBOLSO                       62065.18
Alex Manente         ANDRE SPIGARIOL - SERVIÇOS DE INFORMAÇÃO      DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.             31150.00
FREIRE JÚNIOR        RACHID SOCIEDADE INDIVIDUAL DE ADVOCACIA      CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.    25000.00
Luciano Bivar        ML SERVIÇOS DE COMUNICAÇÃO EIRELI             DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.             21000.00
TAKAYAMA             RACHID SOCIEDADE INDIVIDUAL DE ADVOCACIA      CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.    17000.00
MARCONDES GADELHA    RACHID SOCIEDADE INDIVIDUAL DE ADVOCACIA      CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.    17000.00
Gilberto Nascimento  RACHID SOCIEDADE INDIVIDUAL DE ADVOCACIA      CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.    14000.00
Carlos Veras